In [1]:
import numpy as np
import pandas as pd

In [2]:
orders_df = pd.read_csv("../data/raw/orders.csv")
op_train_df = pd.read_csv("../data/raw/order_products__train.csv")
op_prior_df = pd.read_csv("../data/raw/order_products__prior.csv") 

In [43]:
orders_df.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [46]:
orders_df.eval_set.value_counts() # 75000 for orders in test set

prior    3214874
train     131209
test       75000
Name: eval_set, dtype: int64

In [3]:
# try to get a data fram about the proudcts in each order
def products_concat(series):
    out = ''
    for product in series:
        if product > 0:
            out = out + str(int(product)) + ' '
    
    if out != '':
        return out.rstrip()
    else:
        return 'None'

In [4]:
op_train_df.head()

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


In [7]:
train_product_bought_labels = op_train_df.groupby("order_id")['product_id'].apply(products_concat).reset_index()

In [8]:
train_product_bought_labels.head()

,order_id,product_id
0,1,49302 11109 10246 49683 43633 13176 47209 22035
1,36,39612 19660 49235 43086 46620 34497 48679 46979
2,38,11913 18159 4461 21616 23622 32433 28842 42625...
3,96,20574 30391 40706 25610 27966 24489 39275
4,98,8859 19731 43654 13176 4357 37664 34065 35951 ...


In [35]:
# but our goal is to predict the list of products that would be reordered
def reordered_products_concat(grouped_df):
    out = ''
    for row in grouped_df.itertuples():
        if row[1] > 0 and int(row[2]):
            out = out + str(int(row[1])) + ' '
    
    if out != '':
        return out.rstrip()
    else:
        return 'None'

In [36]:
# get real training labels (reordered products)
train_product_reordered_labels = op_train_df.groupby("order_id")[['product_id', 'reordered']].apply(reordered_products_concat).reset_index()

In [40]:
train_product_reordered_labels.rename(columns = {0: 'products'}, inplace = True)

In [41]:
train_product_reordered_labels.head()

,order_id,products
0,1,49302 11109 43633 22035
1,36,19660 43086 46620 34497 48679 46979
2,38,21616
3,96,20574 40706 27966 24489 39275
4,98,8859 19731 43654 13176 4357 37664 34065 35951 ...


In [72]:
train_product_reordered_labels.query("order_id == 1187899")

,order_id,products
45974,1187899,196 25133 38928 26405 39657 10258 13032 26088 ...


In [90]:
train_df = orders_df.query("eval_set == 'train'")

In [58]:
train_df.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
10,1187899,1,train,11,4,8,14.0
25,1492625,2,train,15,1,11,30.0
49,2196797,5,train,5,0,11,6.0
74,525192,7,train,21,2,11,6.0
78,880375,8,train,4,1,14,10.0


In [52]:
train_product_reordered_labels.shape

(131209, 2)

In [96]:
train_df = pd.merge(train_df, train_product_reordered_labels, on = "order_id" , how = "inner")

In [98]:
train_df.shape

(131209, 8)

In [97]:
train_df.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,products
0,1187899,1,train,11,4,8,14.0,196 25133 38928 26405 39657 10258 13032 26088 ...
1,1492625,2,train,15,1,11,30.0,22963 7963 16589 32792 41787 22825 24852 45066...
2,2196797,5,train,5,0,11,6.0,15349 21413 40706 21616
3,525192,7,train,21,2,11,6.0,47272 37999 13198 43967 40852 17638 29894 45066
4,880375,8,train,4,1,14,10.0,15937 23165 21903 41540


In [99]:
train_df.to_csv("../data/interim/train_orders.csv", index = False)

In [48]:
# prepare the test data frame 
test_df = orders_df.query("eval_set == 'test'")

In [49]:
test_df.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
38,2774568,3,test,13,5,15,11.0
44,329954,4,test,6,3,12,30.0
53,1528013,6,test,4,3,16,22.0
96,1376945,11,test,8,6,11,8.0
102,1356845,12,test,6,1,20,30.0


In [100]:
test_df.to_csv("../data/interim/test_orders.csv", index = False)